<a href="https://colab.research.google.com/github/Ronbragaglia/M-quinas-Virtuais-no-Azure-via-Python/blob/main/M%C3%A1quinas_Virtuais_no_Azure_via_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

!pip install azure-identity azure-mgmt-resource azure-mgmt-compute azure-mgmt-network --quiet

from azure.identity import ClientSecretCredential
from azure.mgmt.resource import ResourceManagementClient
from azure.mgmt.compute import ComputeManagementClient
from azure.mgmt.network import NetworkManagementClient
from azure.mgmt.compute.models import HardwareProfile, NetworkProfile, OSProfile, StorageProfile, ImageReference, OSDisk, DiskCreateOption
import time

SUBSCRIPTION_ID = "SEU_SUBSCRIPTION_ID"
TENANT_ID       = "SEU_TENANT_ID"
CLIENT_ID       = "SEU_CLIENT_ID"
CLIENT_SECRET   = "SEU_CLIENT_SECRET"

RESOURCE_GROUP  = "rg-lab-vm"
LOCATION        = "brazilsouth"
VM_NAME         = "vm-lab-dio"
USERNAME        = "azureuser"
PASSWORD        = "SenhaForte!123"

credential = ClientSecretCredential(
    client_id=CLIENT_ID,
    client_secret=CLIENT_SECRET,
    tenant_id=TENANT_ID
)

resource_client = ResourceManagementClient(credential, SUBSCRIPTION_ID)
network_client  = NetworkManagementClient(credential, SUBSCRIPTION_ID)
compute_client  = ComputeManagementClient(credential, SUBSCRIPTION_ID)

print("Criando Resource Group...")
resource_client.resource_groups.create_or_update(
    RESOURCE_GROUP, {"location": LOCATION}
)

print("Criando Virtual Network...")
vnet = network_client.virtual_networks.begin_create_or_update(
    RESOURCE_GROUP,
    "vnet-lab",
    {
        "location": LOCATION,
        "address_space": {"address_prefixes": ["10.0.0.0/16"]}
    }
).result()

print("Criando Subnet...")
subnet = network_client.subnets.begin_create_or_update(
    RESOURCE_GROUP,
    "vnet-lab",
    "subnet-lab",
    {"address_prefix": "10.0.0.0/24"}
).result()

print("Criando IP Público...")
public_ip = network_client.public_ip_addresses.begin_create_or_update(
    RESOURCE_GROUP,
    "ip-lab",
    {
        "location": LOCATION,
        "public_ip_allocation_method": "Dynamic"
    }
).result()

print("Criando Interface de Rede...")
nic = network_client.network_interfaces.begin_create_or_update(
    RESOURCE_GROUP,
    "nic-lab",
    {
        "location": LOCATION,
        "ip_configurations": [{
            "name": "ipconfig1",
            "subnet": {"id": subnet.id},
            "public_ip_address": {"id": public_ip.id}
        }]
    }
).result()

print("Criando Máquina Virtual...")
vm_params = {
    "location": LOCATION,
    "hardware_profile": {
        "vm_size": "Standard_B1s"
    },
    "storage_profile": {
        "image_reference": {
            "publisher": "MicrosoftWindowsServer",
            "offer": "WindowsServer",
            "sku": "2022-datacenter",
            "version": "latest"
        },
        "os_disk": {
            "create_option": DiskCreateOption.from_string("FromImage"),
            "name": "osdisk-lab"
        }
    },
    "os_profile": {
        "computer_name": VM_NAME,
        "admin_username": USERNAME,
        "admin_password": PASSWORD
    },
    "network_profile": {
        "network_interfaces": [{"id": nic.id}]
    }
}

async_vm_creation = compute_client.virtual_machines.begin_create_or_update(
    RESOURCE_GROUP, VM_NAME, vm_params
)
vm_result = async_vm_creation.result()

print(f"VM '{VM_NAME}' criada com sucesso!")
print(f"IP Público: {public_ip.ip_address} (pode demorar alguns minutos para atribuir)")


